# Programme de Transposition des notes Moodle vers SynapseS


> 
>1. Télécharger les fichiers **Excel** (de Moodle) et **CSV** (de SynapseS) en cliquant sur les boutons correspondants
>2. Transposer les notes lettrées vers SynapseS en cliquant sur le bouton **Traitement**. <br>
    _le séparateur décimal des notes peut être soit , (virgule) soit . (point) selon les besoins de SynapseS_
>3. Récupérer le fichier csv généré en cliquant sur le bouton **'Télécharger xxx.csv'** qui apparaitra.
> 

In [ ]:
import ipywidgets as ipw
from ipywidgets import FileUpload
from IPython.display import display, HTML, clear_output
from IPython.display import FileLink, FileLinks

from ipywidgets import Output, Button
from base64 import b64encode

import pandas as pd
import io,os

def LetterToNum(letter,mode='TP'):
    if mode == 'X':
        # -- X --    
        if letter == 'A+':
            note = 19
        elif letter == 'A':
            note = 17
        elif letter == 'A-':
            note = 15
        elif letter == 'B+':
            note = 14
        elif letter == 'B':
            note = 13
        elif letter == 'B-':
            note = 12
        elif letter == 'C':
            note = 11
        elif letter == 'D':
            note = 9
        elif letter == 'E':
            note = 7
        elif letter == 'F':
            note = 0
        else:
            note = None
    else:
        # -- Telecom --
        if letter == 'A+':
            note = 17.1
        elif letter == 'A':
            note = 16.9
        elif letter == 'A-':
            note = 13.9
        elif letter == 'B+':
            note = 13.9
        elif letter == 'B':
            note = 12.9
        elif letter == 'B-':
            note = 10.9
        elif letter == 'C':
            note = 9.9
        elif letter == 'D':
            note = 7.9
        elif letter == 'E':
            note = 5
        elif letter == 'F':
            note = 0
        else:
            note = None
    return note

def on_upload_change(change):
    #print(change)
    if len(upload1.value)>0 and len(upload2.value)>0:
        button.disabled=False
    
def download_data(e=None):
    #ici lire le fichier csv à envoyer en premier parametre
    with open(cours, 'rb') as f:
        data = f.read().decode('utf-8-sig')
    trigger_download(data, cours, kind='text/csv')

def trigger_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with out:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))
    
def on_button_clicked(a):
    global cours
    with out:
        clear_output()
        
    if upload1.value[list(upload1.value)[0]]['metadata']['type'] != 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        with out:
            display(ipw.Textarea(
            value="Erreur de fichier Moodle, doit être un xls\nLe type actuel est: "+upload1.value[list(upload1.value)[0]]['metadata']['type'],
            description='Erreur:',
            disabled=False,
            layout=ipw.Layout(width="500px")))            
        return
    
    if upload2.value[list(upload2.value)[0]]['metadata']['type'] != 'text/csv':
        with out:
            display(ipw.Textarea(
            value="Le fichier SynapseS n'a pas le format attendu, doit être un xls\nLe type actuel est: "+upload2.value[list(upload2.value)[0]]['metadata']['type'],
            description='Erreur:',
            disabled=False,
            layout=ipw.Layout(width="500px")))
        return

    moodle_file = list(upload1.value.values())[0]
    content1 = moodle_file['content']
    content1 = io.BytesIO(content1)
    dfMoodle = pd.read_excel(content1.read()) 
    output1.value = dfMoodle.head().style.hide_index().to_html() 
    
    synapses_file = list(upload2.value.values())[0]
    content2 = synapses_file['content']
    content2 = io.StringIO(content2.decode('utf-8'))
    dfSynapses = pd.read_csv(content2, delimiter=';', encoding='utf-8-sig',skiprows=1, on_bad_lines='skip')
    output2.value = dfSynapses.head().style.hide_index().to_html()
    
    s1 = dfSynapses['Matricule']
    s2 = dfMoodle['Institution']
    intersection = pd.Series(list(set(s1) & set(s2)))

    if dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)].shape[0] >0:
        output3.value = "<b>Les éléments suivants n'étaient pas dans SynapseS :</b>" + dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)].style.hide_index().to_html()

    for e in intersection:
        dfSynapses.at[dfSynapses['Matricule'] == e,'Note initiale'] = LetterToNum(dfMoodle.loc[dfMoodle['Institution'] == e]['Course letter grade (Real)'].values[0])

    # on écrit le csv
    if cBox.value == True:
        decimal = ','
    else:
        decimal = '.'
    dfSynapses.to_csv('out.csv',sep=';', decimal=decimal, encoding='utf-8', index=False)

    # on lit le fichier original pour récupérer la première ligne et la première colonne
    # On récupère le nom du cours
    content2 = synapses_file['content']
    cours = content2.decode('utf-8-sig').split("\r\n",1)[0].split(';')[0]
    
    # On lit le fichier csv temporaire
    with open("out.csv",'r') as csv:
        csvlines = []
        for line in csv:
            csvlines.append(line)

    # On fusionne les informations
    with open (f'{cours}.csv', 'w', encoding='utf-8-sig') as fp:
        fp.write(content2.decode('utf-8-sig').split("\r\n",1)[0]+"\r\n")
        for l in csvlines:
            fp.write(l)

    # on efface le fichier temporaire
    os.remove('out.csv')

    # on affiche le résultat
    #display(HTML(f'<br>Le fichier resultat à télécharger est : {cours}.csv'),FileLink(f'{cours}.csv'))
    output4.value = f'Le fichier resultat à télécharger est : {cours}.csv<br>' 
    # modifier titre du bouton
    cours = f'{cours}.csv'
    btn.description = f"Télécharger {cours}"
    with out:
        #display(FileLink(cours))
        display(btn)

cours = ''
upload1 = FileUpload(accept='*.xlxs', multiple=False, description='Moodle (.xlsx)', layout=ipw.Layout(width="200px"))
upload2 = FileUpload(accept='*.csv', multiple=False, description='SynapseS (.csv)', layout=ipw.Layout(width="200px"))
button = ipw.Button(description="Traitement", layout=ipw.Layout(width="200px"),  disabled=True)
cBox = ipw.Checkbox(value=True,description="Séparateur décimal ' , '",disabled=False,indent=False)

display(ipw.HBox([upload1,upload2,button,cBox])) 
button.on_click(on_button_clicked)
upload1.observe(on_upload_change, names='_counter')
upload2.observe(on_upload_change, names='_counter')

out = ipw.Output()
btn = Button(description=f'Telecharger {cours}', layout=ipw.Layout(width="600px"))
btn.on_click(download_data)
output1 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%"),  disabled=True)
output2 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%"),  disabled=True)
output3 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%"),  disabled=True)
output4 = ipw.HTML(value="", placeholder="", layout=ipw.Layout(width="100%"),  disabled=True)
accordion = ipw.Accordion(children=[output1,output2,output3,output4], selected_index = None )
accordion.set_title(0,'Moodle')
accordion.set_title(1,'Synapse')
accordion.set_title(2,'Comparaison')
accordion.set_title(3,'Sortie')

display(out,accordion)